In [1]:
#importing the required modules
import os
import numpy as np
import pandas as pd
from scipy.stats import anderson, wilcoxon, ttest_ind

In [2]:
home_dir = os.path.join(r'Z:\balint\numt\Michael\data/')

In [3]:
#read pandas dataframe
numt_array = pd.read_csv(home_dir + 'numt_master_array.csv')
numt_array.head()

,score,eg2_value,e_value,g_id,g_start,mt_start,g_length,mt_length,g_strand,mt_strand,g_size,g_sequence,mt_sequence,g_up_flanking,g_down_flanking,"mt_up_flanking (if strand is negative, the coordinates are corrected)","mt_down_flanking (if strand is negative, the coordinates are corrected)",ensembl_gene_id,ensembl_description
0,17245,0.0,0.0,MT,0,0,17245,17245,+,+,17245,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,NaN,NaN,TACAAATTGTATAATATTTGGACTAAACCCCCTTTCCCACCCCAAG...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,ENSOCUG00000029081,NaN
1,3398,0.0,0.0,1,114731432,6976,4680,4681,+,+,194850757,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,ACAAAAGACTCTGAATAGCCAAAGCGATCCTGAACAAGAAAAATCA...,TATATATATTTATATATATATGTATATATGTACATATATATAACTA...,CCTTCGCCTCAAAACGAGAAGTAGAGACTATTGAACTAACAACCAC...,AACTGCTAATTCATGCTCCCATGTCTAACCCCATGGCTTTCTCAAC...,ENSOCUG00000006104,cilia and flagella associated protein 300 [Sou...
2,3120,0.0,0.0,GL019308,67160,883,4892,4945,+,+,88228,CAAGGCCATGAAGCATGCACACACTGCCCATCGCTCTCCTCAAACA...,CAAGGCCATGAAGCACGCACACACCGCCCGTCACCCTCCTCAAGTG...,GATAAACCCCGATAAACTTTACCACTCTTTGCCAACTCAGCCTCTA...,ATGAAGGGGCAAAGAGTAATTTTACAGTCGAGAGGGCTAGCAGTCA...,TACCGCCATCTTCAGCGAACCCTAAAAAGGAGCAAAAGTAAGCTCA...,TAGGGGCTATTAACTTTATTACAACTATTATTAATATGAAAGCCCC...,ENSOCUG00000031196,NaN
3,1888,0.0,0.0,GL019308,62527,12805,3439,3462,+,+,88228,AATTATTCATAGCCTCAATGATGAACAAGACATCCAAAAAACAGGA...,AATTATTCACAGCCTcaaTGATGAACAGGATATCCGAAAGATAGGC...,GTAGGAATTATATCCTTCCTCCTCATCGGCTGATGGCATGGCCAAG...,ACGTATACATGAGTTGTTCTTACATAATTGAATATACCTGCAATGC...,TGCCTAGGCGCAATCACTACCTTATTCACAGCCCTATGTGCCCTCA...,GTACACCCGTACGCACGCACGTACACCCGTACGCACGCACGTACAC...,ENSOCUG00000037635,NaN
4,1824,0.0,0.0,AAGW02081594,14016,6061,3040,3073,+,+,24576,TTCTTTGGACACCCTGAGGTATATATTCTTATCCTTCCAGGATTCG...,TTTTTCGGGCACCCCGAAGTATATATTCTTATTCTTCCAGGATTTG...,GACATGTCAAAGGAACGCAGAAGCCAGCTTGAAAAAGTCCTCAACA...,TGAAATAGCTATCACTATTCAGGTATGCCACTATTCTTCTAGGCAT...,ATATGAAAGCCCCTGCAATATCTCAATATCAAACCCCCTTATTCGT...,ATTCTCCTAGGTATCTATTTTACCCTACTTCAAGCATCGGAGTATT...,NaN,NaN


In [4]:
#get g_ids and drop the ones that dont have corresponding sequences
g_ids = numt_array['g_id'].drop_duplicates()
g_ids = g_ids.drop(labels = [0,65])

In [5]:
#create dictionary from the numtless sequences
def create_dictionary(g_id):
    global numtless_sequences
    filename = 'numtless_sequences/%s.fasta' % g_id
    sequence = ''
    with open (home_dir + filename) as infile:
        content = infile.readlines()
        try:
            sequence = content[1]
        except IndexError:
            pass
    numtless_sequences[g_id] = sequence
    current_index = g_ids[g_ids == g_id].index[0]
    if current_index % 5 == 0:
        print('done for %s numts!' % current_index)

In [6]:
#create vector for numtless sequences
numtless_sequences = pd.Series(index = g_ids)
g_ids.apply(create_dictionary)
numtless_sequences

C:\Users\pytho\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


done for 20 numts!
done for 25 numts!
done for 40 numts!
done for 45 numts!
done for 70 numts!
done for 95 numts!
done for 100 numts!
done for 105 numts!
done for 125 numts!


g_id
1               AGGACCGCACAACCTTGCAGCTCAGCGACTCGTGGGGTCACACACA...
GL019308        AGCATTTCCAGCATGCAGTCTCTCCCATGAAGGTGGAGAAGGAACT...
AAGW02081594    TTATCTTTATTTCATCCATGATCAAATATTTCACCCACGATGATTA...
GL019326        GAGACTGGGAAGAAGCTCCTGGCTCCTGGCTTTGGATCAGCGCAGC...
GL018741        TAAAACTAAGAGTCATTCTAATCTTTCCTCTATATCTAATCCCAAC...
18              NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
9               CGGGGACTCCCACTAGAGGAACCAGGTGATGTCAGTTCCCAGAATC...
12              CTGAGTTCCTGGCTTCTGGCTTTGGCCTGGCTCAACCCCAGCTGCA...
GL018705        ATAATGATCAGATGATAATGGTGGTGATGATGGTGAGGATGACTCT...
17              GGTGTGGTATATGTGGTGTGTATGGTATGTGTGTGGTACTGTATGG...
GL018700        CCAGATGGATACATGAAGTATGCAGTTGCCTATGTAGTTTAGAACT...
19              NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
GL019465        ATACTTGGCTCCCCACCCAGGAGATCGCAGCTTGAGGCATCACCCC...
GL018817        ACCTAATTACAGGGCACCGGTCTATTTAAAAGATATGTTAAGGGAC...
13              TCATAATCCTAACCATAACCAAGGATTCAACCATAAATTTAAGACT...
GL018

In [7]:
lengths = g_ids.apply(lambda x : numt_array.loc[numt_array['g_id'] == x]['g_length'].tolist())
lengths.index = g_ids
lengths

g_id
1               [4680, 698, 1012, 375, 371, 467, 393, 658, 773]
GL019308                           [4892, 3439, 1414, 837, 820]
AAGW02081594                            [3040, 2339, 1650, 577]
GL019326                     [1851, 1451, 1315, 1420, 995, 833]
GL018741                           [1945, 1590, 1074, 552, 503]
18                                        [1440, 407, 292, 482]
9                                          [720, 822, 397, 273]
12                                              [419, 216, 368]
GL018705                                                  [834]
17                                              [595, 407, 352]
GL018700                                                  [896]
19                                                        [930]
GL019465                              [930, 677, 795, 424, 916]
GL018817                                  [674, 1070, 784, 720]
13                                                   [351, 126]
GL018826              [1193, 891, 8

In [8]:
#calculate gc contents of sequence samples
def gc_content(g_id):
    sequence_to_sample = numtless_sequences[g_id]
    numt_sizes = lengths[g_id]
    gc_contents = []
    for index, numt_size in enumerate(numt_sizes):
        start = np.random.randint(0, (len(sequence_to_sample) - numt_size))
        end = start + numt_size
        sample_sequence = sequence_to_sample[start:end]
        g_content = sample_sequence.upper().count('G')
        c_content = sample_sequence.upper().count('C')
        gc_content = (g_content + c_content) / len(sample_sequence)
        gc_contents.append(gc_content)
    return gc_contents

In [10]:
#calculate gc content of sample sequences
np.random.seed(5)
gc_content_samples = g_ids.apply(gc_content)
gc_content_samples.index = g_ids
gc_content_samples

g_id
1               [0.3767094017094017, 0.0, 0.0, 0.456, 0.401617...
GL019308        [0.4429681112019624, 0.0, 0.43564356435643564,...
AAGW02081594    [0.36414473684210524, 0.5327062847370672, 0.41...
GL019326        [0.3511615343057807, 0.3383873190902826, 0.388...
GL018741        [0.54293059125964, 0.5238993710691824, 0.49627...
18              [0.5069444444444444, 0.43734643734643736, 0.49...
9               [0.3972222222222222, 0.5170316301703163, 0.302...
12              [0.3460620525059666, 0.2824074074074074, 0.505...
GL018705                                     [0.5359712230215827]
17              [0.6016806722689075, 0.37346437346437344, 0.41...
GL018700                                              [0.4296875]
19                                           [0.4043010752688172]
GL019465        [0.37311827956989246, 0.397341211225997, 0.519...
GL018817        [0.6023738872403561, 0.42710280373831777, 0.48...
13                      [0.5555555555555556, 0.40476190476190477]
GL018

In [11]:
#calculate the gc content of numts
def numts_gc(g_id):
    df = numt_array.loc[numt_array['g_id'] == g_id]
    gc_content = df['g_sequence'].apply(lambda x : (x.count('G') + x.count('C')) / len(x.replace('N','')))
    return gc_content.tolist()

In [12]:
#calculate the gc content of numts
numts_gc = g_ids.apply(numts_gc)

In [13]:
numts_gc.index = g_ids
numts_gc

g_id
1               [0.3843689942344651, 0.35704323570432356, 0.36...
GL019308        [0.3726519894970713, 0.38018433179723504, 0.34...
AAGW02081594    [0.387264457439896, 0.3499790180444817, 0.3605...
GL019326        [0.36746666666666666, 0.3604336043360434, 0.36...
GL018741        [0.34470647265429, 0.3409371146732429, 0.31738...
18              [0.3429724277067922, 0.34688995215311, 0.29391...
9               [0.3337856173677069, 0.29832935560859186, 0.36...
12              [0.31116389548693585, 0.3669724770642202, 0.28...
GL018705                                    [0.33568075117370894]
17              [0.39195979899497485, 0.2684085510688836, 0.36...
GL018700                                     [0.2809110629067245]
19                                           [0.3960703205791106]
GL019465        [0.3960703205791106, 0.41594202898550725, 0.34...
GL018817        [0.41884057971014493, 0.3397548161120841, 0.34...
13                       [0.3926553672316384, 0.4126984126984127]
GL018

In [14]:
#function for merging gc contents
def merge(gc_content):
    global sum_gc
    sum_gc += gc_content

In [15]:
#get merged gc content of numts
sum_gc = []
numts_gc.apply(merge)
len(sum_gc)

122

In [16]:
numts_gc = sum_gc
len(numts_gc)

122

In [17]:
sum_gc = []
gc_content_samples.apply(merge)
samples_gc = sum_gc
len(samples_gc)

122

In [18]:
numts_gc

[0.3843689942344651,
 0.35704323570432356,
 0.3617021276595745,
 0.3170103092783505,
 0.3924731182795699,
 0.32238193018480493,
 0.34577114427860695,
 0.3553008595988539,
 0.36014851485148514,
 0.3726519894970713,
 0.38018433179723504,
 0.34599447513812154,
 0.3838973162193699,
 0.38095238095238093,
 0.387264457439896,
 0.3499790180444817,
 0.3605072463768116,
 0.3873720136518771,
 0.36746666666666666,
 0.3604336043360434,
 0.3609865470852018,
 0.34623358673116794,
 0.359,
 0.39766081871345027,
 0.34470647265429,
 0.3409371146732429,
 0.3173873045078197,
 0.32332155477031804,
 0.3672316384180791,
 0.3429724277067922,
 0.34688995215311,
 0.2939189189189189,
 0.3784313725490196,
 0.3337856173677069,
 0.29832935560859186,
 0.36069651741293535,
 0.36996336996337,
 0.31116389548693585,
 0.3669724770642202,
 0.2823834196891192,
 0.33568075117370894,
 0.39195979899497485,
 0.2684085510688836,
 0.36056338028169016,
 0.2809110629067245,
 0.3960703205791106,
 0.3960703205791106,
 0.4159420289855

In [19]:
samples_gc

[0.3767094017094017,
 0.0,
 0.0,
 0.456,
 0.40161725067385445,
 0.3361884368308351,
 0.6030534351145038,
 0.3905775075987842,
 0.34411384217335056,
 0.4429681112019624,
 0.0,
 0.43564356435643564,
 0.37037037037037035,
 0.4170731707317073,
 0.36414473684210524,
 0.5327062847370672,
 0.41393939393939394,
 0.3327556325823224,
 0.3511615343057807,
 0.3383873190902826,
 0.3885931558935361,
 0.528169014084507,
 0.47035175879396984,
 0.4789915966386555,
 0.54293059125964,
 0.5238993710691824,
 0.4962756052141527,
 0.30978260869565216,
 0.39165009940357853,
 0.5069444444444444,
 0.43734643734643736,
 0.4931506849315068,
 0.42946058091286304,
 0.3972222222222222,
 0.5170316301703163,
 0.3022670025188917,
 0.47985347985347987,
 0.3460620525059666,
 0.2824074074074074,
 0.5054347826086957,
 0.5359712230215827,
 0.6016806722689075,
 0.37346437346437344,
 0.41761363636363635,
 0.4296875,
 0.4043010752688172,
 0.37311827956989246,
 0.397341211225997,
 0.519496855345912,
 0.41273584905660377,
 0.525

In [20]:
#normality testing of numts gc
anderson(numts_gc)

AndersonResult(statistic=0.8169277197358014, critical_values=array([0.559, 0.636, 0.763, 0.89 , 1.059]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))

In [21]:
np.median(numts_gc)

0.3554282075772047

In [22]:
#normality testing of samples gc
anderson(samples_gc)

AndersonResult(statistic=4.13250051555525, critical_values=array([0.559, 0.636, 0.763, 0.89 , 1.059]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))

In [23]:
np.median(samples_gc)

0.40977473815555643

In [24]:
wilcoxon(numts_gc, samples_gc)

WilcoxonResult(statistic=1886.0, pvalue=1.8759908422728102e-06)